In [1]:
import _init_paths
import caffe
import numpy as np
import os.path as osp

In [2]:
caffe.set_random_seed(13397)

In [3]:
print osp.abspath(_init_paths.root_dir)

/home/abhijit/Workspace/RenderAndCompare


In [4]:
net = caffe.Net('SoftmaxWithTemperature_Caffe.prototxt', caffe.TEST)

In [5]:
net.blobs['data'].data[...] = np.array([[0, 200, 10], [0, 10, 200], [200, 200, 200], [-1, 0, 1], [1, 1, 0]])
net.blobs['T'].data[...] = np.array([4.0, 1.0, 0.5, 0.1, 0.01]).reshape(-1, 1)

In [6]:
output = net.forward()

In [7]:
print output['softmax_prob_fixedT']

[[ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.33333334  0.33333334  0.33333334]
 [ 0.          0.          1.        ]
 [ 0.5         0.5         0.        ]]


In [8]:
print output['softmax_prob2']

[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]


In [9]:
output['softmax_prob2'].sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [10]:
output['softmax_prob_fixedT'].sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [11]:
fc1_by_T = net.blobs['fc1'].data / net.blobs['T'].data.reshape(-1,1)
assert np.allclose(fc1_by_T, net.blobs['fc1_by_T'].data)

In [22]:
print net.blobs['fc1'].data.shape
print net.blobs['fc1_by_T'].data.shape
print net.blobs['flatT'].data.shape
print net.blobs['T'].data.shape

(5, 1)
(5, 1)
(5, 1)
(5, 1)


In [18]:
net.blobs['fc1_by_T'].data

array([[ 0.05046953],
       [-0.03931031],
       [ 6.00380421],
       [-0.14483018],
       [ 1.52571511]], dtype=float32)

In [13]:
def softmax(x, t=1.0):
    if x.ndim == 1:
        x = x.reshape((1, -1))
    assert x.ndim == 2
    max_x = np.max(x, axis=1).reshape((-1, 1))
    exp_x = np.exp((x - max_x) / t)
    out = exp_x / np.sum(exp_x, axis=1).reshape((-1, 1))
    return out

In [14]:
softmax(net.blobs['fc1_by_T'].data).sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [15]:
output['softmax_prob_fixedT'].sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)